In [56]:
import pandas as pd

data = pd.read_csv('database.csv', encoding='unicode_escape')

criteria = pd.read_csv('criteria.csv')
criteria = criteria.set_index('criteria')

subcrit = pd.read_csv('subcriteria.csv')

category = pd.read_csv('categorization.csv',  encoding='unicode_escape')
category['spesifikasi'] = category['spesifikasi'].map(lambda x: x.lower())
category = category.drop_duplicates(subset=['spesifikasi']).set_index('spesifikasi')

In [59]:
columns = [i for i in data.columns if i in category['criteria'].unique()]

tempData = data.copy()
tempData = tempData[[i for i in data.columns if i in criteria.index]]

for col in columns:
    tempData[col] = data[col].map(lambda val:category.loc[val.lower(),'kelas'])
    

tempData

,brandName,screenSize,screenResolution,processors,diskType,storage,graphicCard,weight,durability,newPrice,secondPrice,ram
0,menengah,13.3,menengah,tinggi,SSD,512,rendah,1.30,5.0,5839000,5500000,16
1,menengah,14.0,menengah,menengah,SSD,256,rendah,1.60,7.5,3500000,3400000,4
2,tinggi,14.2,tinggi,tinggi,SSD,512,tinggi,1.50,11.0,24000000,15700000,16
3,rendah,14.0,menengah,menengah,SSD,256,rendah,1.48,13.0,4000000,3500000,8
4,menengah,15.0,menengah,tinggi,SSD,256,menengah,2.00,53.2,5299000,4900000,8
...,...,...,...,...,...,...,...,...,...,...,...,...
61,menengah,13.0,menengah,menengah,SSD,512,rendah,1.17,16.0,15499000,16499999,8
62,menengah,14.0,rendah,tinggi,SSD,512,menengah,1.60,14.0,3300000,4850000,4
63,menengah,14.0,menengah,menengah,SSD,1000,menengah,1.50,37.0,5400000,4499000,4
64,menengah,13.3,menengah,tinggi,SSD,128,rendah,1.10,12.0,3000000,1840000,4


In [75]:
matrixData = tempData.copy()

for col in tempData.columns:
    if criteria.loc[col, 'weighted']:
        _temp = subcrit[subcrit['criteria'] == col].set_index('value', drop=False)
        if criteria.loc[col,'type'] == 'discrete':
            matrixData[col] = tempData[col].map(lambda x: _temp.loc[x,'weight'])   
        else:
            def process_num(value):
                res = 0
                for index,row in _temp.iterrows():
                    
                    if value >= row['value']:
                        res = row['weight']  
                return res
            
            matrixData[col] = tempData[col].map(process_num)   

matrixData

criteria    durability
weight             0.5
Name: 0, dtype: object
criteria    durability
weight            0.75
Name: 6, dtype: object
criteria    durability
weight             1.0
Name: 12, dtype: object
criteria    durability
weight             0.5
Name: 0, dtype: object
criteria    durability
weight            0.75
Name: 6, dtype: object
criteria    durability
weight             1.0
Name: 12, dtype: object
criteria    durability
weight             0.5
Name: 0, dtype: object
criteria    durability
weight            0.75
Name: 6, dtype: object
criteria    durability
weight             1.0
Name: 12, dtype: object
criteria    durability
weight             0.5
Name: 0, dtype: object
criteria    durability
weight            0.75
Name: 6, dtype: object
criteria    durability
weight             1.0
Name: 12, dtype: object
criteria    durability
weight             0.5
Name: 0, dtype: object
criteria    durability
weight            0.75
Name: 6, dtype: object
criteria    durability
weight 

,brandName,screenSize,screenResolution,processors,diskType,storage,graphicCard,weight,durability,newPrice,secondPrice,ram
0,0.75,13.3,0.75,1.00,1.0,512,0.50,1.30,0,5839000,5500000,16
1,0.75,14.0,0.75,0.75,1.0,256,0.50,1.60,0,3500000,3400000,4
2,1.00,14.2,1.00,1.00,1.0,512,1.00,1.50,0,24000000,15700000,16
3,0.50,14.0,0.75,0.75,1.0,256,0.50,1.48,0,4000000,3500000,8
4,0.75,15.0,0.75,1.00,1.0,256,0.75,2.00,0,5299000,4900000,8
...,...,...,...,...,...,...,...,...,...,...,...,...
61,0.75,13.0,0.75,0.75,1.0,512,0.50,1.17,0,15499000,16499999,8
62,0.75,14.0,0.50,1.00,1.0,512,0.75,1.60,0,3300000,4850000,4
63,0.75,14.0,0.75,0.75,1.0,1000,0.75,1.50,0,5400000,4499000,4
64,0.75,13.3,0.75,1.00,1.0,128,0.50,1.10,0,3000000,1840000,4
